In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline


$F(f(x), g_1(x), ..., g_n(x))$ =  $f(x) + \Sigma_1^n\frac{b_i}{g_i(x)}$

If $b_i$->$0$ then $ argmin(F(x))$ is close to solution of problem:
* minimize_ $(f(x) )$
* subject to_ $(g_i(x) < 0, i=0,..,n) $


In [128]:
def F(f, x, n):
    return (f(x))[0]-(1/(n)/(f(x))[1:]).sum()

Next step is to minimize F(x) using Gradient descent:
* $x_0: g_i(x_0)<0$
* $x_k = x_{k-1} + h*grad_F(x_{k-1})$
* if $|grad_F(x_{k})| < \epsilon$: return $x_k$

In [308]:
# Gradient descent
def mod(x):
    return (x**2).sum()**0.5

def grad(f, x_0):
    L = []
    m = len(x_0)
    for i in range(m):
        step = np.array([0]*i+[0.00001]+[0]*(m-i-1))
        L.append((f(x_0+step)-f(x_0))/0.00001)
    return np.array(L)


def grad_min(f, x_0, N):
    n = 1
    eps = 0.00001
    f_ = lambda x: F(f, x, n)
    g = grad(f_, x_0)
    #print(g, n, eps)
    while mod(g) > eps and n < N:
        n = n+1
        x1 = x_0
        x_0 = x_0 - 0.01*g
        g = grad(f_, x_0)
        #if (f(x_0)[1:]>0).any():
            #return x1
        f_ = lambda x:  F(f, x, n)
    print('steps count = ', n) #for testing
    return x_0
    

In [309]:
# testing function:
def f(x):
    return np.array([(x[0]+2)**2 - x[1], 1-x[1], x[1]-20])

In [310]:
res = grad_min(f, np.array([1, 4]), 10000)
%time
print('result = ', res)

steps count =  10000
CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 26.2 µs
result =  [-2.000005   19.98999475]


Consider discrete case:
* u - parameter - $u \in U = \{u_1, ..., u_n\}$
* $p(u = u_i) = p_i$

In [311]:
# Expected value:
def E(f, x, U, PU):
    def F(u):
        return f(x, u)
    F = np.vectorize(F, signature='()->(n)')
    tmp = F(U)
    res = np.array([0]*len(f(x, U[0])))
    for i in range (len(U)):
        res = res + tmp[i]*PU[i]
    return res

In [358]:
def f_(x_):
    U = np.array([0.5, 1, 2])
    P = np.array([0.2, 0.5, 0.3])
    def f(x, u):
        A = np.array([[2, 1], [1, -1]])
        return (A@x - 1)**2 - 2 + u*x - np.array([u*3, 10])
    return E(f, x_, U, P)

In [363]:
for i in range(5):
    x = grad_min(f_, np.array([1, -2]), 10**i)
    print((f_(x)), x)

steps count =  1
[ -3.4 -10.4] [ 1 -2]
steps count =  10
[ -3.91697784 -10.12823638] [ 1.16379306 -1.86281549]
steps count =  100
[ -4.26379706 -11.58170824] [ 1.0654751  -1.37101856]
steps count =  1000
[-6.09473225 -0.13309079] [-0.44952544  1.6876312 ]
steps count =  10000
[-6.10025357 -0.03523303] [-0.45443354  1.69657795]
